# 자연어처리_뉴스토픽분류 AI 경진대회 

In [ ]:
# !pip install konlpy

In [2]:
# ! pip install lightgbm

## 라이브러리 로드

In [1]:
import pandas as pd
import re
from konlpy.tag import Okt,Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score

from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier

## 데이터 로드

In [2]:
# 데이터 불러오기
train = pd.read_csv('Dataset/train_data.csv')
test = pd.read_csv('Dataset/test_data.csv')

In [3]:
# 데이터 확인
train.tail()

,index,title,topic_idx
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2
45653,45653,2020 한국인터넷기자상 시상식 내달 1일 개최…특별상 김성후,2


In [4]:
# 결측치 확인
train.isnull().sum()

index        0
title        0
topic_idx    0
dtype: int64

In [5]:
# label 비율 확인
train.topic_idx.value_counts()

4    7629
2    7362
5    6933
6    6751
1    6222
3    5933
0    4824
Name: topic_idx, dtype: int64

In [8]:
okt=Okt() 

In [22]:
# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation','Suffix','Number']: #조사, 어미, 구두점 제외 
            clean.append(word[0])
    
    
    return " ".join(clean) 

train['title'] = train['title'].apply(lambda x : func(x))
test['title'] = test['title'].apply(lambda x : func(x))

In [23]:
train['title']

0                   인천 → 핀란드 항공기 결항 휴가 철 여행객 분통
1                     실리콘밸리 넘어서다 구글 들이다 美 전역 거점
2               이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3        NYT 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리 종합
4                     시진핑 트럼프 중미 무역 협상 조속 타결 희망
                          ...                  
45649             KB 금융 미국 IB 스티 펠 제휴 선진국 시장 공략
45650           보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토
45651              게시판 키우다 증권 키우다 영웅 전 실전 투자 대회
45652                      답변 하다 배 기동 국립 중앙 박물관
45653               한국 인터넷 기자 시상식 내달 개최 특별상 김성후
Name: title, Length: 45654, dtype: object

In [27]:
# 불용어 리스트 생성
stopwords = ['의', '가', '이', '은', '들', '는','…...','·','→','되다','때','시','보','전','첫','간','안',
             '좀', '잘', '걍', '과', '도', '을','를', '명','돼다','곳','듯','속',
             '...','.',',','…','종합','·','냐','것', '→','↑','위','급','회','각','후','내',
             '로','으로', '자', '에', '와', '한', '하다']

In [28]:
X_train = [] 
for sentence in train['title']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거 
    X_train.append(temp_X) 
    
X_test = [] 
for sentence in test['title']: 
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거 
    X_test.append(temp_X)

In [29]:
X_train

[['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통'],
 ['실리콘밸리', '넘어서다', '구글', '들이다', '美', '전역', '거점'],
 ['이란', '외무', '긴장', '완화', '해결', '책', '미국', '경제', '전쟁', '멈추다'],
 ['NYT', '클린턴', '측근', '韓', '기업', '특수', '관계', '조명', '공과', '사', '맞다', '물리'],
 ['시진핑', '트럼프', '중미', '무역', '협상', '조속', '타결', '희망'],
 ['팔레스타인', '가자지구', '세', '소년', '이스라엘군', '총격', '사망'],
 ['인도', '파키스탄', '공습', '테러', '캠프', '폭격'],
 ['美',
  '대선',
  'TV',
  '토론',
  '음담패설',
  '만회',
  '실패',
  '트럼프',
  '사과',
  '대신',
  '빌다',
  '클린턴',
  '공격',
  '역효과'],
 ['푸틴', '한반도', '상황', '진전', '방안', '김정은', '위원장', '논의'],
 ['특검', '면죄부', '받다', '트럼프', '스캔들', '보도', '언론', '맹공', '국민', '적'],
 ['日', '오키나와', '열', '리다', '강제', '징용', '노동자', '추도'],
 ['이란', '최고', '지도자', '모욕', '혐의', '미국인', '징역', '선고'],
 ['카니발', '축제', '보다', '가다', '브라질', '리우', '대형', '유람선', '행렬'],
 ['美', '올랜도', '병원', '최악', '총기', '테러', '부상', '치료', '비', '받다'],
 ['日', '대기업', '올해', '평균', '임금', '인상'],
 ['WMO', '엘니뇨', '여전하다', '강력', '기', '소멸'],
 ['이스라엘', '네타냐후', '유대교도', '병역', '문제', '연정', '협상', '진통'],
 ['UAE', '사우

In [30]:
from keras.preprocessing.text import Tokenizer 
max_words = 35000 
tokenizer = Tokenizer(num_words = max_words) 
tokenizer.fit_on_texts(X_train) 
X_train = tokenizer.texts_to_sequences(X_train) 
X_test = tokenizer.texts_to_sequences(X_test)


In [31]:
import numpy as np 
y_train = [] 

for i in range(len(train['topic_idx'])): 
    if train['topic_idx'].iloc[i] == 0: 
        y_train.append([1, 0, 0, 0, 0, 0, 0]) 
    elif train['topic_idx'].iloc[i] == 1: 
        y_train.append([0, 1, 0, 0, 0, 0, 0]) 
    elif train['topic_idx'].iloc[i] == 2: 
        y_train.append([0, 0, 1, 0, 0, 0, 0]) 
    elif train['topic_idx'].iloc[i] == 3: 
        y_train.append([0, 0, 0, 1, 0, 0, 0]) 
    elif train['topic_idx'].iloc[i] == 4: 
        y_train.append([0, 0, 0, 0, 1, 0, 0]) 
    elif train['topic_idx'].iloc[i] == 5: 
        y_train.append([0, 0, 0, 0, 0, 1, 0]) 
    elif train['topic_idx'].iloc[i] == 6: 
        y_train.append([0, 0, 0, 0, 0, 0, 1]) 
        
y_train = np.array(y_train)


In [32]:
from keras.layers import Embedding, Dense, LSTM 
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences 
max_len = 20 # 전체 데이터의 길이를 20로 맞춘다 
X_train = pad_sequences(X_train, maxlen=max_len) 
X_test = pad_sequences(X_test, maxlen=max_len)


In [ ]:
model = Sequential() 
model.add(Embedding(max_words, 100)) 
model.add(LSTM(128)) 
model.add(Dense(7, activation='softmax')) 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)


Epoch 1/10
4109/4109 [==============================] - 264s 63ms/step - loss: 0.8698 - accuracy: 0.6891 - val_loss: 0.4824 - val_accuracy: 0.8401
Epoch 2/10
1667/4109 [===========>..................] - ETA: 2:25 - loss: 0.2456 - accuracy: 0.9182

In [62]:
predict = model.predict(X_test)
topic = []
for i in range(len(predict)):
    topic.append(np.argmax(predict[i]))

## 재출 데이터 생성

In [65]:
submission = pd.read_csv('Dataset/sample_submission.csv')

In [28]:
submission['topic_idx'] = pred
submission.tail(10)

,index,topic_idx
9121,54775,0
9122,54776,2
9123,54777,2
9124,54778,2
9125,54779,2
9126,54780,3
9127,54781,6
9128,54782,4
9129,54783,0
9130,54784,2


In [67]:
submission.to_csv('submission/LSTM_0708.csv',index = False)